In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
import torch 
from NCF_model.NCF import *
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re

In [8]:
ratings_df = pd.read_csv("cleaned_100k/ratings.csv")
Movies_df=  pd.read_csv("cleaned_100k/descriptions_df.csv")
Movies_df.drop(columns=['Unnamed: 0'],inplace=True)

title = pd.read_csv("cleaned_100k/movies.csv")
content_df  =  pd.merge(ratings_df,Movies_df,on='MovieID')
content_df.drop(columns=['Timestamp'],inplace=True)
#X =  ratings_df.drop(columns=['Rating'])
#y = ratings_df['Rating']
#content_df  =   pd.read_csv("cleaned_100k/content_df.csv")

y_group_user =  content_df.groupby('UserID')
y_target_content = []
for i,j  in  y_group_user:
    y_target_content.append(j['Rating'].values)
x_group_user =  content_df.groupby('UserID')
#content_df.drop(columns=['Ratings'],inplace=True)
x_train_content = []
for i,j  in  x_group_user:
    x_train_content.append(j.drop(columns='UserID'))
    
    
n_users,n_movies,n_factors =610,9742,50
model = NCF(n_users,n_movies,n_factors).to('cuda')
model.load_state_dict(torch.load('NCF_model/model.pth', weights_only=True))
model.eval()

NCF(
  (user_factors): Embedding(610, 50)
  (movie_factors): Embedding(9742, 50)
  (layers): Sequential(
    (0): ReLU()
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=100, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=1, bias=True)
  )
)

In [ ]:
new_val = {'Users': 68,'Recommender_Ratings':2}
Recommender_Response1 = Recommender_Response1.append(new_val, ignore_index = True)
Recommender_Response1.to_csv('Recommender_Responsev1.csv') 

C:\Users\sendh\AppData\Local\Temp\ipykernel_38560\1937313586.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Recommender_Response1 = Recommender_Response1.append(new_val, ignore_index = True)


In [33]:
Recommender_Response1

,Users,Recommender_Ratings
0,5.0,6.0


In [21]:
Recommender_Response1._append??

Signature:
Recommender_Response1._append(
    other,
    ignore_index: 'bool' = False,
    verify_integrity: 'bool' = False,
    sort: 'bool' = False,
) -> 'DataFrame'
Docstring: <no docstring>
Source:   
    def _append(
        self,
        other,
        ignore_index: bool = False,
        verify_integrity: bool = False,
        sort: bool = False,
    ) -> DataFrame:
        if isinstance(other, (Series, dict)):
            if isinstance(other, dict):
                if not ignore_index:
                    raise TypeError("Can only append a dict if ignore_index=True")
                other = Series(other)
            if other.name is None and not ignore_index:
                raise TypeError(
                    "Can only append a Series if ignore_index=True "
                    "or if the Series has a name"
                )

            index = Index([other.name], name=self.index.name)
            row_df = other.to_frame().T
            # infer_objects is needed for
          

In [52]:
def Hybrid_Recommender(user_id):
    watched = set(x_train_content[user_id]['MovieID'].values)
    not_watched = Movies_df[Movies_df['MovieID'].map(lambda x: x not in watched)]
    result = pd.DataFrame({'Movie_id':not_watched['MovieID']})
    CBF(user_id,not_watched,result)
    NCF(user_id,not_watched,result)
    result['hybrid_preds'] =  result['collab_preds'] * 0.5 + result['content_preds'] *0.5 
    
    return result

In [53]:
def  CBF(user_id,not_watched,result):
    #train  =  [ast.literal_eval(x) for x  in   x_train_content[user_id]['TFIDF']]
    train = x_train_content[user_id].iloc[:,2:]
    model =  SVR(C=1.3)
    model.fit(train,y_target_content[user_id])
  
    not_watched_tfidf = not_watched.iloc[:,:-1]
    preds = model.predict(not_watched_tfidf)
    preds =  np.clip(preds,0,5)
    result['content_preds'] = preds

In [54]:
def NCF(user_id,not_watched,result):
    movies = LabelEncoder()
    movies.classes_ = np.load('movies.npy')
    users = LabelEncoder()
    users.classes_ = np.load('users.npy')
    x_predict_collab = torch.stack((torch.tensor(users.transform([4]* len(not_watched))),torch.tensor(movies.transform(not_watched['MovieID']))),1).to('cuda')
    model.eval()
    preds = model(x_predict_collab)
    result['collab_preds'] = preds.to('cpu').tolist()

In [55]:
def Top_10_Recommendations(user_id):
    result = Hybrid_Recommender(user_id)
    movie_ids = result.sort_values('hybrid_preds', ascending=False)['Movie_id'][:10]
    movies = list(title[title['MovieID'].isin(movie_ids)]['Title'].values)
    return movies

In [135]:
def Similar_movies(name):
    cosine_sim = cosine_similarity(Movies_df.iloc[:,:-1],Movies_df.iloc[:,:-1])
    idxs = title[title['Title'] == name].index[0]
    sim_scores = list(enumerate(cosine_sim[idxs]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    similar = [x  for x,y in sim_scores]
    return list(title.iloc[similar]['Title'])

In [57]:
Top_10_Recommendations(1)

['Perfect Candidate, A (1996)',
 'Cruise, The (1998)',
 'Slasher (2004)',
 "Darwin's Nightmare (2004)",
 'Connections (1978)',
 'All Watched Over by Machines of Loving Grace (2011)',
 'Century of the Self, The (2002)',
 'The Blue Planet (2001)',
 'Planet Earth (2006)',
 'Blue Planet II (2017)']

In [56]:
Top_10_Recommendations(600)

['Raging Bull (1980)',
 '7th Voyage of Sinbad, The (1958)',
 'Woman in the Dunes (Suna no onna) (1964)',
 "Man Bites Dog (C'est arrivé près de chez vous) (1992)",
 'Trial, The (Procès, Le) (1962)',
 'Come and See (Idi i smotri) (1985)',
 'Sword of Doom, The (Dai-bosatsu tôge) (1966)',
 'Bittersweet Life, A (Dalkomhan insaeng) (2005)',
 'Incendies (2010)',
 'Paddington 2 (2017)']

In [58]:
Top_10_Recommendations(444)

['His Girl Friday (1940)',
 'Young Frankenstein (1974)',
 'Place in the Sun, A (1951)',
 'All That Heaven Allows (1955)',
 'Belle époque (1992)',
 'Trial, The (Procès, Le) (1962)',
 'Eddie Murphy Delirious (1983)',
 'Incendies (2010)',
 'Everybody Wants Some (2016)',
 'Logan (2017)']

In [138]:
Similar_movies('Raging Bull (1980)')

['Nymphomaniac: Volume II (2013)',
 'Passion of the Christ, The (2004)',
 'Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)',
 'Infinity (1996)',
 'Mr. Holmes (2015)',
 'Tristram Shandy: A Cock and Bull Story (2005)',
 'Louis C.K.: Hilarious (2010)',
 'Wit (2001)',
 'No Way Jose (2015)',
 'W. (2008)']

In [178]:
res = Hybrid_Recommender(555)

In [179]:
res['content_preds'].max(),res['collab_preds'].max()

(4.521811722104937, 4.293727397918701)

In [150]:
Hybrid_Recommender(356).sort_values('hybrid_preds', ascending=False)['Movie_id'][:10]

3265      4419
2011      2677
2424      3224
2411      3201
8241    104283
2582      3451
7863     93988
7602     86781
5407     25771
6473     52767
Name: Movie_id, dtype: int64

In [133]:
mov = result.sort_values('hybrid_preds', ascending=False)['Movie_id'][:10]
list(title[title['MovieID'].isin(mov)]['Title'].values)

['Star Wars: Episode IV - A New Hope (1977)',
 'Raging Bull (1980)',
 'Trust (1990)',
 'Manhattan (1979)',
 'Manchurian Candidate, The (1962)',
 'Woman in the Dunes (Suna no onna) (1964)',
 'Dog Soldiers (2002)',
 'Bittersweet Life, A (Dalkomhan insaeng) (2005)',
 'Raiders of the Lost Ark: The Adaptation (1989)',
 'All-Star Superman (2011)']

In [130]:
result.sort_values('hybrid_preds', ascending=False)

,Movie_id,content_preds,collab_preds,hybrid_preds
929,1228,4.085381,4.187244,6.229934
7063,69524,4.529487,3.883759,6.148503
936,1236,3.788333,4.215914,6.110080
2424,3224,3.664027,4.266088,6.098102
4083,5833,3.837443,4.176753,6.095474
...,...,...,...,...
2105,2799,3.492524,1.550392,3.296654
2964,3973,3.495651,1.525869,3.273694
2825,3774,3.517409,1.509495,3.268199
5180,8387,3.327863,1.581333,3.245265


# Rough Work

In [39]:
Recommender_Response1 = pd.read_csv('Recommender_Responsev1.csv',usecols=['Users','Recommender_Ratings'])  
Recommender_Response2 = pd.read_csv('Recommender_Responsev2.csv',usecols=['Users','Recommender_Ratings'])

In [38]:
from scipy.stats import ttest_ind

In [42]:
ttest_ind(Recommender_Response1['Recommender_Ratings'].values,Recommender_Response2['Recommender_Ratings'].values)

Ttest_indResult(statistic=0.1346845820953142, pvalue=0.8941438046418504)

In [52]:
ttest_ind([1,2,1,0,1,2],[5,4,5,3,4,5],alternative='greater')

Ttest_indResult(statistic=-6.984538909891173, pvalue=0.999981072080732)

In [44]:
ttest_ind?

Signature:
ttest_ind(
    a,
    b,
    axis=0,
    equal_var=True,
    nan_policy='propagate',
    permutations=None,
    random_state=None,
    alternative='two-sided',
    trim=0,
)
Docstring:
Calculate the T-test for the means of *two independent* samples of scores.

This is a test for the null hypothesis that 2 independent samples
have identical average (expected) values. This test assumes that the
populations have identical variances by default.

Parameters
----------
a, b : array_like
    The arrays must have the same shape, except in the dimension
    corresponding to `axis` (the first, by default).
axis : int or None, optional
    Axis along which to compute test. If None, compute over the whole
    arrays, `a`, and `b`.
equal_var : bool, optional
    If True (default), perform a standard independent 2 sample test
    that assumes equal population variances [1]_.
    If False, perform Welch's t-test, which does not assume equal
    population variance [2]_.

    .. versionadde

In [ ]:
'''
Recommender_Response1 = pd.DataFrame({'Users':[],'Recommender_Ratings':[]})
Recommender_Response2 = pd.DataFrame({'Users':[],'Recommender_Ratings':[]})
                    
Recommender_Response1.to_csv('Recommender_Responsev1.csv') 
                                                         
Recommender_Response2.to_csv('Recommender_Responsev2.csv')     

new_val = {'Users': 68,'Recommender_Ratings':2}
Recommender_Response1 = Recommender_Response1.append(new_val, ignore_index = True)
Recommender_Response1.to_csv('Recommender_Responsev1.csv') 
'''

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
cosine_sim = cosine_similarity(Movies_df.iloc[:,:-1],Movies_df.iloc[:,:-1])

In [82]:
idxs = title[title['Title'] == 'Toy Story (1995)'].index[0]

In [122]:
sim_scores = list(enumerate(cosine_sim[idxs]))

In [123]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:21]

In [126]:
similar = [x  for x,y in sim_scores]

In [127]:
#similar = list(np.argsort(cosine_sim[idxs])[:10])

In [132]:
list(title.iloc[similar]['Title'])

["Bug's Life, A (1998)",
 'Turbo (2013)',
 'Toy Story 2 (1999)',
 'Monsters, Inc. (2001)',
 'Antz (1998)',
 'The Lego Movie (2014)',
 'Shrek (2001)',
 'Asterix and the Vikings (Astérix et les Vikings) (2006)',
 'Gnomeo & Juliet (2011)',
 'Moana (2016)',
 'Over the Hedge (2006)',
 'Space Jam (1996)',
 'Happy Feet (2006)',
 'Paddington 2 (2017)',
 'Rio 2 (2014)',
 'Ant Bully, The (2006)',
 'Puss in Boots (Nagagutsu o haita neko) (1969)',
 'The Good Dinosaur (2015)',
 'Karlson Returns (1970)',
 "Porky's Hare Hunt (1938)"]

In [77]:
Movies_df[Movies_df['MovieID'] == 193581].index[0]

9737

In [4]:
movies = LabelEncoder()
movies.classes_ = np.load('movies.npy')
users = LabelEncoder()
users.classes_ = np.load('users.npy')

In [8]:
user_id =77
watched = set(x_train_content[user_id]['MovieID'].values)
not_watched = Movies_df[Movies_df['MovieID'].map(lambda x: x not in watched)]

In [36]:
x_predict_collab = torch.stack((torch.tensor(users.transform([4]* len(not_watched))),torch.tensor(movies.transform(not_watched['MovieID']))),1)

In [37]:
n_users,n_movies,n_factors =610,9742,50
model = NCF(n_users,n_movies,n_factors)
model.load_state_dict(torch.load('NCF_model/model.pth', weights_only=True))
model.eval()

NCF(
  (user_factors): Embedding(610, 50)
  (movie_factors): Embedding(9742, 50)
  (layers): Sequential(
    (0): ReLU()
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=100, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=1, bias=True)
  )
)